## Importing packages and defining functions 

In [74]:
import dask as da
from dask.distributed import LocalCluster, Client
import datetime
from datetime import date 
from datetime import datetime
import glob
import scipy.ndimage as ndimage
from scipy.ndimage.measurements import label, find_objects
import xarray as xr
local_dir = "/g/data/e14/cp3790/dask-workers"
cluster = LocalCluster(processes=False, local_dir=local_dir)
client = Client(cluster)
client

Client Scheduler: inproc://10.0.64.14/13389/46 Dashboard: http://10.0.64.14/13389/46:46132/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


In [84]:
# This function will make the clim and thresh repeatable so that the shapes of clim, thresh and obs are equal.

def fix(ds):
    # This was the code fixed to make the clim and thresh repeatable so that the shapes of clim, thresh and obs are equal.
    trial = ds
    i = 0
    while i < 37:  #<-- Here, 37 corresponds to the number of years in obs dataset 
        trial = xr.concat([trial, ds], 'dayofyear')
        i+=1
    trial = trial.isel(dayofyear = slice(0,13514))
    # to specify the dates/time 
    trial.coords['dayofyear'] = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1) 
    
    # This code was used to rename the dayofyear dimension to time.
    trial['time'] = trial['dayofyear']
    del trial['dayofyear']
    trial = trial.rename({'dayofyear': 'time'})
    
    t = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1)
    dates = [date.fromordinal(tt.astype(int)) for tt in t]
    
    trial.coords['time'] = dates

    return trial

In [76]:
def consec(temps):
    labels, _ = label(temps) # labels the occurrence of non-zero values (in this case, 1) and gives it an 'event' number 
    slices = find_objects(labels) 
    new_temps = np.zeros(len(temps))
    
    for i in slices:
        if temps[i].size >= 3:
            new_temps[i] = temps[i]
        else:
            new_temps[i] = 0
    return new_temps

In [77]:
def consec_ufunc(array, axis):
    result = np.apply_along_axis(
        consec, 
        axis,
        array
    )
    return result 

In [78]:
def consecutive(dataarray):
    result = xr.apply_ufunc(
        consec_ufunc, dataarray,
        input_core_dims=[['time']],
        output_core_dims=[['time']],
        kwargs={'axis':-1},
        output_sizes={'time':len(dataarray['time'])}
    )
    return result.transpose(*dataarray.dims)

## Opening files 

In [79]:
mean_climatology = xr.open_dataarray('/g/data/e14/cp3790/Charuni/climatology-australia.nc')
threshold = xr.open_dataarray('/g/data/e14/cp3790/Charuni/threshold-australia.nc')

In [80]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/ERA5-new/era5_dailytmax_*.nc'))

era5_dailytmax_aus = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1982', '2018'), longitude=slice(113, 154), latitude=slice(-10, -44))

obs_aus_tmax = era5_dailytmax_aus['dmax'].load()
obs_aus_tmax.attrs['units'] = 'deg C'

## Calculations

In [85]:
new_climatology = fix(mean_climatology)

In [86]:
new_threshold = fix(threshold)

In [87]:
threshold_anomaly = new_threshold - new_climatology

In [88]:
dailytmax_anomaly = obs_aus_tmax - new_climatology

In [89]:
thw_severity = dailytmax_anomaly/threshold_anomaly

In [90]:
hws_preDur = xr.where(thw_severity > 0, thw_severity, 0)

In [91]:
oscar = consecutive(hws_preDur)

distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [94]:
oscar.to_netcdf('/g/data/e14/cp3790/Charuni/thw-severity.nc')

MemoryError: Unable to allocate array with shape (13514, 137, 165) and data type float64